# To build a recommender system that matches users for a trade based on their wanted and selling list

In [ ]:
#  %cd /content/drive/MyDrive/CMPE_256_Project
#  ! git clone https://github.com/swapit/code.git

## Required Imports

In [ ]:
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn.model_selection import train_test_split
import random

/home/015266823/venv/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
print(pd.__version__)

1.1.5


# Preprocessing

In [ ]:
DATA_PATH = "/home/015266823/bookmooch/bm1y"

def read_data_into_dfs(path):
  have_df = pd.read_csv(f"{DATA_PATH}/have.csv", names=["user_id", "item_id", "unix_timestamp"])
  wish_df = pd.read_csv(f"{DATA_PATH}/wish.csv", names=["user_id", "item_id", "unix_timestamp"])
  pairs_df = pd.read_csv(f"{DATA_PATH}/pairs.csv", names=["user1_id", "item1_id", "user2_id", "item2_id"])
  transac_df = pd.read_csv(f"{DATA_PATH}/transac.csv", names=["giver_user_id", "receiver_user_id", "item_id", "unix_timestamp"])
  return have_df, wish_df, pairs_df, transac_df

have_df, wish_df, pairs_df, transac_df = read_data_into_dfs(DATA_PATH)

In [ ]:
have_df.head(1)

,user_id,item_id,unix_timestamp
0,41,415868,1421536572


In [ ]:
wish_df.head(1)

,user_id,item_id,unix_timestamp
0,41,12889,1425062294


In [ ]:
pairs_df.head(1)

,user1_id,item1_id,user2_id,item2_id
0,80555,84914,5616,495048


In [ ]:
transac_df.head(1)

,giver_user_id,receiver_user_id,item_id,unix_timestamp
0,38,15502,354627,1416308635


### Prepare User-Item Matrix

In [ ]:
def user_item_df(have_df, wish_df, pairs_df, transac_df):
  # Grab all the items the user has wished for
  wished_for = wish_df[['user_id',	'item_id']]

  # Get items from the transactions.
  transacted = transac_df[['receiver_user_id', 'item_id']].rename({'receiver_user_id': 'user_id'}, axis='columns')

  # Combine the wished for and transaction dataframe
  user_item = pd.concat([wished_for, transacted])

  # Add a like column indicating they have liked it.
  user_item['like'] = 1

  # These are the duplicate entries (that is, when someone got something they wished for)
  print(f'Number of user_items before removing duplicates: {user_item.shape[0]}')
  print(f'Number of duplicated rows: {user_item[user_item.duplicated()].shape[0]}')
  user_item = user_item.drop_duplicates()
  print(f'Number of duplicated rows after removing: {user_item[user_item.duplicated()].shape[0]}')
  print(f'Number of user_items after removing duplicates: {user_item.shape[0]}')

  return user_item

In [ ]:
user_item = user_item_df(have_df, wish_df, pairs_df, transac_df)

Number of user_items before removing duplicates: 546785
Number of duplicated rows: 8613
Number of duplicated rows after removing: 0
Number of user_items after removing duplicates: 538172


In [ ]:
user_item.user_id.describe()

count    538172.000000
mean      43788.941558
std       26092.649412
min          41.000000
25%       21341.000000
50%       44447.000000
75%       66695.000000
max      124671.000000
Name: user_id, dtype: float64

### Change Ids To Sequence

In [ ]:
def change_ids_to_seq(user_item):
  # Create user and item mapping to convert to seq
  user_ids = user_item['user_id'].unique()
  user_ids.sort()

  item_ids = user_item['item_id'].unique()
  item_ids.sort()

  user_id_map = {real: mapped for mapped, real in enumerate(user_ids)}
  item_id_map = {real: mapped for mapped, real in enumerate(item_ids)}

  def find_user_id_mapping(user_id, user_id_map):
    return user_id_map[user_id]
  
  def find_item_id_mapping(item_id, item_id_map):
    return item_id_map[item_id]

  user_item["user_id"] = user_item.apply(lambda s: find_user_id_mapping(s["user_id"], user_id_map), axis=1)
  user_item["item_id"] = user_item.apply(lambda s: find_item_id_mapping(s["item_id"], item_id_map), axis=1)
  return user_item, user_id_map, item_id_map

In [ ]:
user_item=user_item.head(300000)
# user_item=user_item.head(230000)
user_item, user_id_map, item_id_map = change_ids_to_seq(user_item)

### Train-Test Split

In [ ]:
def split_into_test_train(user_item):
  train_df, test_df = train_test_split(user_item, test_size=0.2)

  test_df_zreos = test_df.copy()
  test_df_zreos["like"] = 0
  train_df = train_df.append(test_df_zreos)
  # train_df[train_df["like"] == 0]

  return train_df, test_df

In [ ]:
train_df, test_df = split_into_test_train(user_item)

In [ ]:
train_df

,user_id,item_id,like
293457,8487,23180,1
46066,1077,37066,1
58288,1501,86630,1
230444,6788,6196,1
244868,7180,123798,1
...,...,...,...
259634,7604,193848,0
239698,6974,147846,0
255103,7469,191476,0
41436,924,54977,0


In [ ]:
test_df

,user_id,item_id,like
222945,6515,123038,1
299877,8617,207233,1
298520,8576,110031,1
93304,2604,114003,1
55995,1399,84031,1
...,...,...,...
259634,7604,193848,1
239698,6974,147846,1
255103,7469,191476,1
41436,924,54977,1


# Modeling Basic User Preferences





In [ ]:
def get_train_matrix(train_df):
  # Get the training matrix with 0s entered for all user-item combos.
  train_matrix_df = train_df.pivot(index='user_id', columns ='item_id', values = 'like').fillna(0)
  return train_matrix_df

In [ ]:
matrix_df = get_train_matrix(train_df)

In [ ]:
matrix_df

item_id,0,1,2,3,4,5,6,7,8,9,...,207320,207321,207322,207323,207324,207325,207326,207327,207328,207329
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8615,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
matrix = matrix_df.to_numpy()

In [ ]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

The first goal of our model is to estimate a user’s preference for an individual item. As our data contains wish lists
and past transactions, we use them as implicit feedback signals [18] when building the preference model.


Following the approach proposed by Hu et al. [10], the
user-item interaction matrix R is built based on implicit
feedback signals as follows:

(i.e., 1 iff the item belongs to uj ’s wish list, or there exists
a past transaction in which uj receives ik).

where puj and qik
are vectors describing the ‘preferences’ of
the user uj and the ‘properties’ of the item ik.


In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(matrix, k = 50) 
sigma = np.diag(sigma)


In [ ]:
predicted_user_items = np.dot(np.dot(U, sigma), Vt)
predicted_user_items_df = pd.DataFrame(predicted_user_items, columns = matrix_df.columns)

In [ ]:
predicted_user_items_df = predicted_user_items_df.rename_axis('user_id').rename_axis("item_id", axis="columns")
predicted_user_items_df

item_id,0,1,2,3,4,5,6,7,8,9,...,207320,207321,207322,207323,207324,207325,207326,207327,207328,207329
user_id,,,,,,,,,,,,,,,,,,,,,
0,3.372824e-07,8.931657e-06,0.0,-1.322453e-05,6.836703e-07,-4.774879e-05,9.474291e-04,0.0,9.664367e-06,8.933580e-07,...,4.560785e-06,4.560785e-06,0.0,4.560785e-06,4.560785e-06,4.560785e-06,4.560785e-06,4.560785e-06,4.560785e-06,4.560785e-06
1,-2.095458e-07,1.651866e-06,0.0,1.585755e-07,4.262733e-08,4.773616e-06,3.066345e-05,0.0,4.380769e-06,2.194858e-08,...,1.940328e-08,1.940328e-08,0.0,1.940328e-08,1.940328e-08,1.940328e-08,1.940328e-08,1.940328e-08,1.940328e-08,1.940328e-08
2,3.231701e-08,3.403137e-06,0.0,-1.052859e-06,2.073240e-07,-4.843679e-06,-6.111273e-06,0.0,5.015399e-06,-2.931927e-08,...,-3.845174e-06,-3.845174e-06,0.0,-3.845174e-06,-3.845174e-06,-3.845174e-06,-3.845174e-06,-3.845174e-06,-3.845174e-06,-3.845174e-06
3,5.642433e-20,1.039158e-19,0.0,6.121800e-18,-3.424745e-20,2.521348e-18,-2.284601e-17,0.0,5.362164e-19,-5.929335e-20,...,1.656322e-17,1.656322e-17,0.0,1.656322e-17,1.656322e-17,1.656322e-17,1.656322e-17,1.656322e-17,1.656322e-17,1.656322e-17
4,-1.040540e-20,-1.242097e-20,0.0,-7.970409e-19,4.295882e-21,-1.867968e-19,2.968654e-18,0.0,-6.934653e-20,7.721410e-21,...,-2.204643e-18,-2.204643e-18,0.0,-2.204643e-18,-2.204643e-18,-2.204643e-18,-2.204643e-18,-2.204643e-18,-2.204643e-18,-2.204643e-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8613,1.042698e-07,1.590465e-06,0.0,-2.254642e-06,5.710171e-07,-2.101926e-06,7.640194e-04,0.0,8.890386e-07,7.526058e-07,...,-1.594632e-07,-1.594632e-07,0.0,-1.594632e-07,-1.594632e-07,-1.594632e-07,-1.594632e-07,-1.594632e-07,-1.594632e-07,-1.594632e-07
8614,2.679520e-06,4.367085e-06,0.0,-5.588498e-06,8.007883e-07,7.088423e-06,8.367701e-04,0.0,8.663724e-06,9.202548e-07,...,6.105635e-06,6.105635e-06,0.0,6.105635e-06,6.105635e-06,6.105635e-06,6.105635e-06,6.105635e-06,6.105635e-06,6.105635e-06
8615,1.226905e-06,1.680613e-05,0.0,-1.404506e-06,2.829148e-06,-6.478610e-05,-1.237403e-05,0.0,5.865253e-05,-9.430550e-08,...,-5.782663e-06,-5.782663e-06,0.0,-5.782663e-06,-5.782663e-06,-5.782663e-06,-5.782663e-06,-5.782663e-06,-5.782663e-06,-5.782663e-06


## Social Bias

#### Map Social friendship/Trust between users using their Transaction History

In [ ]:
#calculating total number of transactions between user1 and user2

#Using Dict, key=(giver_user_id,receiver_user_id) ,value = total number of transactions between giver_user_id and receiver_user_id .
#return: It is returning dict which contains all the number of transactions between giver and reciever. 


def transactions_U1_U2(transac_df):
  
  socialfactorU1_U2_Dict={}
  
  for i,row in transac_df.iterrows():
    # print(i['giver_user_id'])
    key=(row['giver_user_id'],row['receiver_user_id'])
    if key in socialfactorU1_U2_Dict:
      
      socialfactorU1_U2_Dict[key]=socialfactorU1_U2_Dict[key]+1
    else:
      
      socialfactorU1_U2_Dict[key]=1
  # print(tempkey)
  return socialfactorU1_U2_Dict

In [ ]:
socialfactorU1_U2_Dict=transactions_U1_U2(transac_df)

In [ ]:
len(socialfactorU1_U2_Dict)

109628

#### Map All the users in Have_df to retrieve the users having a particular Item Id

In [ ]:
#In this we are retrieving a list of all the users having a specific item in the form of {"itemid":["user1","user2"],"itemid2":["user3"] }from have_df.
#after this we will call recommend a user from above list who is having that item and high social bias score for that particular user.

#haveDict is the dictionary for all the users having that item
def retrieveAllUsersHavingThatItem(have_df):
    
    haveDict = {}

    tempKey = 0
    for i,row in have_df.iterrows():
        # print(i['giver_user_id'])
        value=row['user_id']
        key=row['item_id']
        if key in haveDict:
            if value not in haveDict[key]:
                haveDict[key].append(value)
        else:
            haveDict[key] = [value]
    return haveDict

In [ ]:
UsersHavingThatItem_map=retrieveAllUsersHavingThatItem(have_df)

In [ ]:
len(UsersHavingThatItem_map)

65678

#### SocialActivity Factor counts the number of transactions an individual User has done based on his past transactions

In [ ]:
#Calculating the value of transactionCountFactor to check if the user is socially Active?
#Adding the sociallyActiveUserFactor to the result of social bias to get the socialScore


#Transaction Count 
def getSocialActivityFactor(transac_df,pairs_df):
    transactionCount = {}
    max_variable = 0
    #Transactions from transac_df
    for i,row in transac_df.iterrows():
        # print(i['giver_user_id'])
        key1 = row['giver_user_id']
        key2 = row['receiver_user_id']

        if key1 in transactionCount:
            transactionCount[key1] = transactionCount[key1]+1
            max_variable = max(max_variable,transactionCount[key1])
        else:
            transactionCount[key1] = 1

        if key2 in transactionCount:
            transactionCount[key2] = transactionCount[key2]+1
            max_variable = max(max_variable,transactionCount[key2])
        else:
            transactionCount[key2] = 1
            
    #Transactions from pairs_df
    for i,row in pairs_df.iterrows():
        # print(i['giver_user_id'])
        key1 = row['user1_id']
        key2 = row['user2_id']

        if key1 in transactionCount:
            transactionCount[key1] = transactionCount[key1]
            max_variable = max(max_variable,transactionCount[key1])
        else:
            transactionCount[key1] = 1
        if key2 in transactionCount:
            transactionCount[key2] = transactionCount[key2]
            max_variable = max(max_variable,transactionCount[key2])
        else:
            transactionCount[key2] = 1
#     print (max_variable) 
    return transactionCount,max_variable

In [ ]:
socialActivityDict,maxTransaction= getSocialActivityFactor(transac_df,pairs_df)

#### Social Bias Score gives the normalised score based on the User Activity and User-User Transactions

In [ ]:
def socialBiasScore(userId,itemId):

    giverId = 0
    maximumScore = 0
    if itemId in UsersHavingThatItem_map:
        for i in UsersHavingThatItem_map[itemId]:
            try:
                maximumScore = max(maximumScore,socialfactorU1_U2_Dict[i][userId])
                
                giverId = i
            except:
                try:
                    maximumScore = max(maximumScore,socialfactorU1_U2_Dict[userId][i])
                    giverId = i
                except:
                    pass
    
    
    if maximumScore == 0:
        if userId in socialActivityDict:
            return (socialActivityDict[userId]/len(socialActivityDict))
        
        return 0
    

    social_bias_score=  (maximumScore/maxTransaction)    
    print(maxTransaction)   
    return social_bias_score

# Evaluation 

In [ ]:
def print_full(x):
  pd.set_option('display.max_rows', len(x))
#   print(x)
  pd.reset_option('display.max_rows')


def get_test_test_triplates(predicted_user_items_df, test_df, user_item, user_id_map, item_id_map,socialBias):

  def find_predicted_for_liked_item(user_id, item_id):
    
    socialBiasValue = socialBiasScore(user_id,item_id)
   
    if socialBias:
        return predicted_user_items_df.iloc[user_id][item_id].item() + socialBiasValue
        #return sum(predicted_user_items_df.iloc[user_id][item_id].item() ,float(socialBiasScore(user_id,item_id,have_df,socialBiasFactorDict)))
    else:
        return predicted_user_items_df.iloc[user_id][item_id].item()
        

  def get_random_item_for_user(user_id):
    random_item_id = random.randrange(0, len(item_id_map)) 

    liked_item_ids_for_user = set(user_item[user_item["user_id"] == user_id]["item_id"].values)
    if not random_item_id in liked_item_ids_for_user:
      return random_item_id

    while random_item_id in liked_item_ids_for_user:
      random_item_id = random.randrange(0, len(item_id_map))
    return random_item_id

  def find_predicted_for_random_item(user_id, item_id):
#     print(user_id)
#     print(item_id)
    return predicted_user_items_df.iloc[int(user_id)][int(item_id)].item()

  
  evaluation_df = test_df[["user_id",	"item_id"]].rename({'item_id': 'liked_item_id'}, axis='columns')
  evaluation_df["prediction_for_liked"] = evaluation_df.apply(lambda s: find_predicted_for_liked_item(s["user_id"], s["liked_item_id"]), axis=1)
  evaluation_df["random_item_id"] = evaluation_df.apply(lambda s: get_random_item_for_user(s["user_id"]), axis=1)
  evaluation_df["prediction_for_random"] = evaluation_df.apply(lambda s: find_predicted_for_random_item(s["user_id"], s["random_item_id"]), axis=1)

  return evaluation_df

In [ ]:
evaluation_df = get_test_test_triplates(predicted_user_items_df, test_df, user_item, user_id_map, item_id_map,False)


In [ ]:
evaluation_df_social = get_test_test_triplates(predicted_user_items_df, test_df, user_item, user_id_map, item_id_map,True)


In [ ]:
evaluation_df.head()

,user_id,liked_item_id,prediction_for_liked,random_item_id,prediction_for_random
222945,6515,123038,0.0,124383,-8.001543e-13
299877,8617,207233,0.0,52492,1.797751e-03
298520,8576,110031,0.0,169275,6.287296e-20
93304,2604,114003,0.0,72734,1.981969e-04
55995,1399,84031,0.0,91298,0.000000e+00


In [ ]:
def evaluation_func(evaluation_df):
  def get_difference(liked_item_pred, random_item_pred):
    diff = liked_item_pred - random_item_pred
    if diff > 0:
      return 1
    else:
      return 0
  evaluation_df["difference"] = evaluation_df.apply(lambda s: get_difference(s["prediction_for_liked"], s["prediction_for_random"]), axis=1)
  return evaluation_df["difference"].sum()/evaluation_df["difference"].count()

In [ ]:
from scipy.sparse.linalg import svds


for n in range(50, 500, 50):
	print(f"N: {n}")
  U, sigma, Vt = svds(matrix, k = n)
  sigma = np.diag(sigma)


  predicted_user_items = np.dot(np.dot(U, sigma), Vt)
  predicted_user_items_df = pd.DataFrame(predicted_user_items, columns = matrix_df.columns)
  predicted_user_items_df = predicted_user_items_df.rename_axis('user_id').rename_axis("item_id", axis="columns")

	evaluation_df = get_test_test_triplates(predicted_user_items_df, test_df, user_item, user_id_map, item_id_map,False)
	e = evaluation_func(evaluation_df)
	print(f"Social Bias: False E: {e}")

	evaluation_df_social = get_test_test_triplates(predicted_user_items_df, test_df, user_item, user_id_map, item_id_map,True)
	e = evaluation_func(evaluation_df_social)
 
	print(f"Social Bias: True E: {e}")

In [ ]:
e_1 = evaluation_func(evaluation_df) #nfactors=100 


In [ ]:
print(e_1)

0.5412333333333333


In [ ]:
e1 = evaluation_func(evaluation_df_social)  #nfactors=100

In [ ]:
print(e1)

0.6167833333333334


In [ ]:
# e_1 = evaluation_func(evaluation_df) #nfactors=50 with best result


In [ ]:
qprint(e_1)

NameError: name 'qprint' is not defined

In [ ]:
e1 = evaluation_func(evaluation_df_social)  #nfactors=50 with best result

In [ ]:
print(e1)

In [ ]:
e = evaluation_func(evaluation_df) #nfactors=50

In [ ]:
print(e)

In [ ]:
e_1 = evaluation_func(evaluation_df) #nfactors=50


In [ ]:
print(e_1)

In [ ]:
e1 = evaluation_func(evaluation_df_social)  #nfactors=50

In [ ]:
print(e1)

In [ ]:
e2 = evaluation_func(evaluation_df_social)  #nfactors=50

In [ ]:
print(e2)

In [ ]:
e_3 = evaluation_func(evaluation_df)  #nfactors=50


In [ ]:
print(e_3)

In [ ]:
e3 = evaluation_func(evaluation_df_social) #nfactors=50

In [ ]:
print(e3)

In [ ]:
e_4 = evaluation_func(evaluation_df) #nfactors=50


In [ ]:
print(e_4)

In [ ]:
e4 = evaluation_func(evaluation_df_social)  #nfactors=50

In [ ]:
print(e4)

In [ ]:
e_5 = evaluation_func(evaluation_df) #nfactors=1000


In [ ]:
print(e_5)

In [ ]:
e5 = evaluation_func(evaluation_df_social) #nfactors=1000

In [ ]:
print(e5)

In [ ]:
e_6 = evaluation_func(evaluation_df) #nfactors=100


In [ ]:
print(e_6)

In [ ]:
e6 = evaluation_func(evaluation_df_social) #nfactors=100

In [ ]:
print(e6)

In [ ]:
e_7 = evaluation_func(evaluation_df) #nfactors=10


In [ ]:
print(e_7)

In [ ]:
e7 = evaluation_func(evaluation_df_social) #nfactors=10

In [ ]:
print(e7)

In [ ]:
e_8 = evaluation_func(evaluation_df) #nfactors=200


In [ ]:
print(e_8)

In [ ]:
e8 = evaluation_func(evaluation_df_social) #nfactors=200

In [ ]:
print(e8)

In [ ]:
e_8 = evaluation_func(evaluation_df) #nfactors=150


In [ ]:
print(e_8)

In [ ]:
e8 = evaluation_func(evaluation_df_social) #nfactors=150

In [ ]:
print(e8)

In [ ]:
e_8 = evaluation_func(evaluation_df) #nfactors=80


In [ ]:
print(e_8)

In [ ]:
e8 = evaluation_func(evaluation_df_social) #nfactors=80

In [ ]:
print(e8)

In [ ]:
e_8 = evaluation_func(evaluation_df) #nfactors=500


In [ ]:
print(e_8)

In [ ]:
e8 = evaluation_func(evaluation_df_social) #nfactors=500

In [ ]:
print(e8)

In [ ]:
e_9 = evaluation_func(evaluation_df) #nfactors=5


In [ ]:
print(e_9)

In [ ]:
e9 = evaluation_func(evaluation_df_social) #nfactors=5

In [ ]:
print(e9)

In [ ]:
e_9 = evaluation_func(evaluation_df) #nfactors=250


In [ ]:
print(e_9)

In [ ]:
e9 = evaluation_func(evaluation_df_social) #nfactors=250

In [ ]:
print(e9)

In [ ]:
e_9 = evaluation_func(evaluation_df) #nfactors=2000


In [ ]:
print(e_9)

In [ ]:
e9 = evaluation_func(evaluation_df_social) #nfactors=2000